# "ExoDoc" - Deep Learning for Tumor Detection and Identification in Exo Travellers

ExoDoc is a CNN (Convolutional Neural Network) created to detect and classify brain tumors in MRI scans of the pioneering spacefarers of the 22nd century. It is built using Tensorflow and Keras, and has been trained on 3000 MRI scans of space explorers to detect 4 distinct classes of tumors.

ExoDoc was built by Caramel Labs as a submission for the Datathon of the Tech-Triathlon, organized by Rootcode.

## Getting Started

In [1]:
# Test Python runtime

print('Hello world!')

Hello world!


In [2]:
# Tensorflow imports

# 1. Tensorflow
import tensorflow as tf

# 2. Neural network architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras.regularizers import l2

2023-08-25 04:12:18.797243: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-25 04:12:18.803138: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-25 04:12:18.897590: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-25 04:12:18.899193: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 04:12:21.593720: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [4]:
# Other imports

import os
import shutil
from sklearn.model_selection import train_test_split

## The Data

Our dataset is a collection of 3000 brain MRI scans, each being 512 x 512 px. They have been taken from various angles and portray various depths of the human brain. The dataset is composed of 4 distinct classes:

1. Category 1 tumor
2. Category 2 tumor
3. Category 3 tumor
4. No tumor

Here's what the current file structure of the data looks like:

Separating the images into training and testing datasets will be beneficial for us. We can execute a simple Python script to achieve this.

In [ ]:
# Automatically move images into training and testing folders

def split_data_into_folders():
    # Path to original dataset folder
    original_data_path = './data'

    # Path to create the new train and test folders
    train_data_path = './data/train'
    test_data_path = './data/test'

    # Create train and test folders if they don't exist
    os.makedirs(train_data_path, exist_ok=True)
    os.makedirs(test_data_path, exist_ok=True)

    # List of category folders
    categories = ['category1_tumor', 'category2_tumor', 'category3_tumor', 'no_tumor']

    # Iterate through each category
    for category in categories:
        category_path = os.path.join(original_data_path, category)
        images = os.listdir(category_path)
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

        # Create subdirectories for each category in train and validation folders
        os.makedirs(os.path.join(train_data_path, category), exist_ok=True)
        os.makedirs(os.path.join(test_data_path, category), exist_ok=True)

        # Move images to train and validation folders
        for img in train_images:
            src = os.path.join(category_path, img)
            dest = os.path.join(train_data_path, category, img)
            shutil.copy(src, dest)

        for img in test_images:
            src = os.path.join(category_path, img)
            dest = os.path.join(test_data_path, category, img)
            shutil.copy(src, dest)

split_data_into_folders()

Our data now looks like:

Since we have a relatively small dataset (in the scale of datasets usually used to train neural networks), we will augment the dataset with `ImageDataGenerator`. This allows the model to receive new variations of its training images for each epoch.

In [5]:
# Data augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Instantiate ImageDataGenerator with augmentation settings
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

Next, we will use the `flow_from_directory()` method of `ImageDataGenerator` to automatically label the data based on the folder structure we created previously.

In [6]:
# Data loading and preprocessing

batch_size = 32

train_generator = datagen.flow_from_directory(
    './data/train',
    target_size=(256, 256),  # Resize images as necessary
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    './data/test',
    target_size=(256, 256),  # Resize images as necessary
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2577 images belonging to 4 classes.
Found 647 images belonging to 4 classes.


## The Neural Network

It is common knowledge within the circles of ML enthusiasts that CNNs (Convolutional Neural Networks) are renown for their performance in image analysis. So, we decided to build ExoDoc using a custom-built CNN (instead of pre-trained as they are not permitted by the rules of the Datathon). After multiple iterations, endless research papers and countless hours on Kaggle, using inspiration from the architectures of models created by other ML practitioners, we built our CNN using the following architecture:

In [7]:
# Defining the model architecture

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    GlobalAveragePooling2D(),
    Dropout(0.5),

    Dense(128, activation='relu'),
    Dropout(0.2),

    Dense(64, activation='relu'),
    Dropout(0.2),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(4, activation='softmax')
])

In [8]:
# Getting a summary of the model

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 256, 256, 32)      128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 256, 256, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 32)      9248      
                                                                 
 batch_normalization_1 (Bat  (None, 256, 256, 32)      128       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 256, 256, 32)      0

We can use callbacks to hook into various stages of the training process to ensure that overfitting is minimized.

In [7]:
# Defining callbacks

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

early_stop = EarlyStopping(monitor='loss',
                           patience=5,
                           verbose = 1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=5,
                              verbose=1)

In [10]:
# Compiling the model

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

After compiling the model, we can start training. We selected a relatively high number of epochs to reach the highest possible accuracy before reaching a plateau. We also included the previously defined callbacks.

In [11]:
# Training the Neural Network

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=40,  # Adjust the number of epochs as needed
    validation_data=test_generator,
    callbacks=[early_stop, reduce_lr],
    validation_steps=test_generator.samples // batch_size,
    verbose=1
)

Epoch 1/40
80/80 [==============================] - 158s 2s/step - loss: 1.3583 - accuracy: 0.3399 - val_loss: 1.3722 - val_accuracy: 0.2781 - lr: 0.0010
Epoch 2/40
80/80 [==============================] - 153s 2s/step - loss: 1.1624 - accuracy: 0.4483 - val_loss: 1.3668 - val_accuracy: 0.2766 - lr: 0.0010
Epoch 3/40
80/80 [==============================] - 152s 2s/step - loss: 1.1045 - accuracy: 0.5053 - val_loss: 1.3636 - val_accuracy: 0.2844 - lr: 0.0010
Epoch 4/40
80/80 [==============================] - 151s 2s/step - loss: 1.0272 - accuracy: 0.5532 - val_loss: 1.3678 - val_accuracy: 0.3344 - lr: 0.0010
Epoch 5/40
80/80 [==============================] - 152s 2s/step - loss: 0.9859 - accuracy: 0.5859 - val_loss: 1.9402 - val_accuracy: 0.3359 - lr: 0.0010
Epoch 6/40
80/80 [==============================] - 151s 2s/step - loss: 0.9116 - accuracy: 0.6322 - val_loss: 2.4327 - val_accuracy: 0.3359 - lr: 0.0010
Epoch 7/40
80/80 [==============================] - 151s 2s/step - loss: 0.8

Finally, we can check how well our model performs against testing data.

In [12]:
# Evaluating the loss

test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print("Test loss: ", test_loss)
print("Test accuracy: ", test_acc)

21/21 [==============================] - 15s 695ms/step - loss: 0.3472 - accuracy: 0.8748
Test loss:  0.34719160199165344
Test accuracy:  0.874806821346283


Our final accuracy is a satisfactory 87.48%. However, we will continue to improve this by further hyperparameter fine-tuning and rethinking the model's architecture.

We can now export our model to be used in production.

In [ ]:
# Serializing the model

model.save('mri_classification_model_v3.h5')